In [2]:
import pandas as pd 
import numpy as np

In [11]:
df = pd.read_csv('../Dataset\HeartDisease.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [12]:
df_num = df.select_dtypes(np.number)
df=df_num

In [13]:
inputs=df.drop('HeartDisease',axis='columns')
target=np.array(df.HeartDisease)

In [14]:
from sklearn.model_selection import train_test_split
X_train,Xtest,Y_train,Y_test=train_test_split(inputs,target,test_size=0.3)

In [15]:
from sklearn.neighbors import KNeighborsClassifier
knn =KNeighborsClassifier(n_neighbors=7)

In [16]:
knn.fit(X_train,Y_train)
y_pred=knn.predict(Xtest)

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [17]:
print(y_pred)
len(y_pred)

[1 0 0 1 0 1 1 0 0 0 0 1 1 0 0 0 1 1 1 0 1 1 1 1 0 0 0 1 1 0 1 0 1 0 0 1 0
 0 0 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 0
 1 1 0 0 1 1 1 0 1 0 1 1 1 1 1 1 0 1 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 1 0 0 0
 1 0 1 1 1 0 0 1 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0
 0 1 0 1 0 0 1 0 1 0 1 0 1 0 0 1 1 0 0 0 0 1 1 1 0 1 1 0 0 1 1 0 1 1 1 1 1
 1 1 0 0 0 1 0 1 1 0 1 1 0 0 1 0 1 1 1 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 1
 1 0 0 1 1 0 1 0 0 0 1 1 1 1 0 0 0 1 1 0 1 1 1 0 1 0 0 0 1 1 0 1 1 0 1 0 1
 0 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0]


276

In [18]:
from sklearn import metrics
ac= metrics.accuracy_score(Y_test,y_pred)
print(ac)


0.7065217391304348


In [22]:
from  sklearn.metrics  import classification_report ,confusion_matrix
print(classification_report(Y_test,y_pred))
print(confusion_matrix(Y_test,y_pred))


              precision    recall  f1-score   support

           0       0.66      0.71      0.68       123
           1       0.75      0.71      0.73       153

    accuracy                           0.71       276
   macro avg       0.70      0.71      0.70       276
weighted avg       0.71      0.71      0.71       276

[[ 87  36]
 [ 45 108]]
